In [11]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [12]:
# Create retriever tool for wikipedia
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [13]:
wiki.name

'wikipedia'

In [14]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/") 
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
embeddings=OllamaEmbeddings(model="llama3")
vectordb=Chroma.from_documents(documents, embeddings)
retriever=vectordb.as_retriever()
retriever

USER_AGENT environment variable not set, consider setting it to identify your requests.


VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001B440CADA30>)

In [15]:
# Create retriever tool for website
from langchain.tools.retriever import create_retriever_tool
langsmith=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [16]:
langsmith.name

'langsmith_search'

In [17]:
# Create retriever tool for Arxiv
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [18]:
tools=[wiki,arxiv,langsmith]

In [19]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Projects\\LangChain\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001B429A60A60>, retriever=VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorsto

In [25]:
from dotenv import load_dotenv
load_dotenv()
import os
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
llm=ChatOllama(model="llama3")

In [26]:
# Agents
from langchain import hub
# Get the prompt to use - you can modify this!
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [27]:
# Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [28]:
# Agent executor
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(...)
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'}

In [29]:
agent_executor.invoke({"input":"Tell me about langsmith"})



> Entering new AgentExecutor chain...
Langsmith!

Langsmith is an online platform that allows you to create, customize, and share your own language learning flashcards. The platform is designed for individuals who want to learn new languages or improve their vocabulary in their native tongue.

Here are some key features of Langsmith:

1. **Customization**: You can create your own flashcard sets with words, phrases, or sentences that you find most relevant to your learning goals.
2. **Vocabulary building**: Langsmith offers a vast library of user-generated content, allowing you to discover and learn new words in various languages.
3. **Interactive exercises**: The platform includes interactive exercises, such as matching games, word searches, and crossword puzzles, to help you practice and reinforce your language skills.
4. **Collaboration**: You can join communities or create your own groups to connect with other learners, share tips, and learn from each other's experiences.
5. **Off

{'input': 'Tell me about langsmith',
 'output': "Langsmith!\n\nLangsmith is an online platform that allows you to create, customize, and share your own language learning flashcards. The platform is designed for individuals who want to learn new languages or improve their vocabulary in their native tongue.\n\nHere are some key features of Langsmith:\n\n1. **Customization**: You can create your own flashcard sets with words, phrases, or sentences that you find most relevant to your learning goals.\n2. **Vocabulary building**: Langsmith offers a vast library of user-generated content, allowing you to discover and learn new words in various languages.\n3. **Interactive exercises**: The platform includes interactive exercises, such as matching games, word searches, and crossword puzzles, to help you practice and reinforce your language skills.\n4. **Collaboration**: You can join communities or create your own groups to connect with other learners, share tips, and learn from each other's exp

In [30]:
agent_executor.invoke({"input":"What's the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...
Paper 1605.08386!

According to my research, this is an arXiv preprint titled "Learning to Simulate Dynamic Scenes" by researchers from the University of California, Berkeley and the University of Toronto.

In a nutshell, this paper presents a novel approach to learning to simulate dynamic scenes, such as those found in movies or video games. The authors propose a generative model that can synthesize realistic videos of objects moving through space.

The key innovation is a hierarchical architecture that combines a physics-based simulation with a neural network-based refinement module. This allows the model to learn to predict complex movements and interactions between objects, while also capturing subtle details like texture and lighting effects.

The authors demonstrate their approach on various scenarios, including robotic manipulation tasks and virtual character animations. Their results show impressive gains in realism and diversity compared

{'input': "What's the paper 1605.08386 about?",
 'output': 'Paper 1605.08386!\n\nAccording to my research, this is an arXiv preprint titled "Learning to Simulate Dynamic Scenes" by researchers from the University of California, Berkeley and the University of Toronto.\n\nIn a nutshell, this paper presents a novel approach to learning to simulate dynamic scenes, such as those found in movies or video games. The authors propose a generative model that can synthesize realistic videos of objects moving through space.\n\nThe key innovation is a hierarchical architecture that combines a physics-based simulation with a neural network-based refinement module. This allows the model to learn to predict complex movements and interactions between objects, while also capturing subtle details like texture and lighting effects.\n\nThe authors demonstrate their approach on various scenarios, including robotic manipulation tasks and virtual character animations. Their results show impressive gains in re